In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
import json
import matplotlib.pyplot as plt

# Clear
清理不符合要求的数据，并删除对应的图片和ground_truth文件。

In [ ]:
group_path = 'group1'

input_path = os.path.join('./data_origin/picwithgt', group_path)
target_gt_path = os.path.join('./data_origin/newgt', group_path)
target_pic_path = os.path.join('./data_origin/newpic', group_path)

In [ ]:
files = []
for file in os.listdir(input_path): 
    file_name = file.split('-')[1].split('.')[0]
    files.append(file_name)

print(len(files))

In [ ]:
num = 0
for file in os.listdir(target_gt_path): 
    file_name = file.split('-')[1].split('.')[0]
    if file_name not in files:
        os.remove(os.path.join(target_gt_path, file))
        print(file)
        num += 1
print(num)

In [ ]:
num = 0
for file in os.listdir(target_pic_path): 
    file_name = file.split('-')[1].split('.')[0]
    if file_name not in files:
        os.remove(os.path.join(target_pic_path, file))
        print(file)
        num += 1
print(num)

# Images process
- 原始数据中存在着部分图片的分辨率为1920*1080.
- 部分数据虽然分辨率为1280*1024，但是四周存在着黑框，对训练会有影响。
- 将原始图片以及检测框的ground_truth缩放到512*512大小

In [ ]:
TARGET_W = 1280
TARGET_H = 1024

pic_train_path = '../data/train'
pic_test_path = '../data/test'

annos_path = '../data/annotation_type5.json'
annos = json.load(open(annos_path, 'r'))

In [ ]:
def annos_resize(imgid, old_W, old_H, new_W=1280, new_H=1024):
    print(imgid, old_W, old_H)
    x_ratio = old_W / new_W
    y_ratio = old_H / new_H
    objects = annos['imgs'][imgid]['objects']
    for bid, bboxs in enumerate(objects):
        bbox = bboxs['bbox']
        print('origin: ', bbox)
        bbox['xmin'] = bbox['xmin'] / x_ratio
        bbox['xmax'] = bbox['xmax'] / x_ratio
        bbox['ymin'] = bbox['ymin'] / y_ratio
        bbox['ymax'] = bbox['ymax'] / y_ratio
        print('resize: ', bbox)
        annos['imgs'][imgid]['objects'][bid]['bbox'] = bbox
        

In [ ]:
def data_clear(path):
    num = 0
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
#         img = cv2.imread(file_path)
        img = Image.open(file_path)
        img = np.array(img)
        if img.ndim == 2:
            print(file)
            os.remove(file_path)
            num += 1
    print('total resize num: ', num)

In [ ]:
data_clear(pic_train_path)

In [ ]:
data_clear(pic_test_path)

In [ ]:
file_path = os.path.join(pic_train_path, 'group4-31_1_2.jpg')
# img = cv2.imread(file_path)
img = Image.open(file_path)
img = np.array(img)

print(img.ndim)
plt.imshow(img)

# Check
测试resize之后的检测框框是否正常

In [ ]:
imgid = 'group1-282'
img = cv2.imread('./data_tmp/train/'+imgid+'.jpg')

objects = annos['imgs'][imgid]['objects']
for bid, bboxs in enumerate(objects):
    bbox = bboxs['bbox']
    category = bboxs['category']
    x_min = int(bbox['xmin'])
    x_max = int(bbox['xmax'])
    y_min = int(bbox['ymin'])
    y_max = int(bbox['ymax'])
    print(category, x_min, x_max, y_min, y_max)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    cv2.putText(img, category, (int(x_min), int(y_min - 6)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (255, 255, 255))

plt.imshow(img)
plt.show()

# Sample
从Tsinghua_tencent_100K中，每个类别随机选择1000张图片加入训练集中

In [ ]:
import numpy as np
import os
import json
import shutil

In [ ]:
annos_path = '../data/type3/annotation_3class.json'
train_path = '../data/type3/train'
pic_output_path = '../data/type3/samples'
annos_output_path = '../data/type3/annotation_sample.json'

In [ ]:
annos = json.load(open(annos_path, 'r'))
if os.path.exists(pic_output_path):
    shutil.rmtree(pic_output_path)
    os.mkdir(pic_output_path)

In [ ]:
cnt = {'z':0, 'j':0, 's':0}
num = 1000

annos_new={}
annos_new['imgs']={}

for file in os.listdir(train_path):
    imgid = file.split('.')[0]
    objects = annos['imgs'][imgid]['objects']
    for bid, bboxs in enumerate(objects):
        category = bboxs['category']
        if cnt[category] < num:
            print(file)
            cnt[category] += 1
            shutil.copy(train_path+'/'+file, pic_output_path+'/')
            annos_new['imgs'][imgid] = annos['imgs'][imgid]
            break

In [ ]:
cnt

In [ ]:
with open(annos_output_path, 'w') as f:
    json.dump(annos_new, f)
    f.close()

# Data Set Partitioning
从数据集中随机划分1000张到测试集

In [ ]:
import os
import json
import random
import shutil

In [ ]:
ipic_dist = {}
train_path = '../data/train/'
test_path = '../data/test/'
data_num = 0
test_num = 1000
for i, ipic in enumerate(os.listdir(test_path)):
    ipic_dist[i] = ipic
    data_num += 1
#     print(i, ipic)
# print(data_num)

In [ ]:
rand_list = []
for i in range(test_num):
    ran = random.randint(0, data_num-1)
    while ran in rand_list:
        ran = random.randint(0, data_num-1)
    rand_list.append(ran)

In [ ]:
for i in rand_list:
    try:
        shutil.move(train_path + ipic_dist[i], test_path + ipic_dist[i])
        print(ipic_dist[i])
    except:
        pass

# Resize
将所有测试集resize到512*512大小

In [21]:
import os
import cv2
import numpy as np
from PIL import Image

In [22]:
WIDTH = 512
HEIGHT = 512

test_path = '../data/final_test'
output_path = '../data/final_test_resize'

assert os.path.exists(test_path)
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [24]:
for file in os.listdir(test_path):
#     img = Image.open(os.path.join(test_path, file))
    img = cv2.imread(os.path.join(test_path, file))
    height, width = img.shape[:2]
    print(file, width, height)
    if width != WIDTH or height != HEIGHT:
        img = cv2.resize(img, (WIDTH, HEIGHT))
        cv2.imwrite(os.path.join(output_path, file), img)

group3-470.png 1174 938
group3-494.png 1174 938
group3-486.png 1174 938
group3-474.png 1174 938
group3-479.png 1174 938
